In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np
import math
mpl.rcParams['figure.dpi']= 300
sns.set(rc={'text.usetex' : True, 'figure.autolayout': True})
import sys
sys.path.append("..")
import src.constants as cnst
import src.opt as runner
from typing import Tuple, List

In [ ]:
DATA_PATH = cnst.DATA_PATH
all_cols = cnst.all_col_path
FIG_PATH = cnst.FIG_PATH
df = pd.read_csv(DATA_PATH + all_cols)
df.rename(columns={'perc_below_2pov': 'PctPvty'}, inplace=True)

In [ ]:
df.describe()

### Optimization Algorithm

In [ ]:
from sklearn.linear_model import LinearRegression
def opt(x: np.ndarray, green_pct: np.ndarray, ar: np.ndarray, pop: np.ndarray, pvty_pct: np.ndarray, funds: int, injection: float, reg: LinearRegression) -> Tuple[np.ndarray, float, List]:
    """
    Optimize the utility function for geographic tracts
    Returns geometric mean
    """
    util_delta = []
    util_array = x
    while funds > 0:
        total_util = runner.geo_mean(util_array)
        max_util_delta, max_util_idx = 0, -1
        for idx, u in enumerate(util_array):
            temp = util_array.copy()
            new_green = runner.green_change(injection, green_pct[idx], ar[idx])
            if new_green > 95:
                continue
            else:
                temp[idx] += runner.util(pop[idx], green_pct[idx], new_green, pvty_pct[idx], reg)
                curr_util = runner.geo_mean(temp)
                delta = curr_util - total_util
                if delta >= max_util_delta:
                    max_util_delta = delta
                    max_util_idx = idx
        util_delta.append(total_util)
        ng = runner.green_change(injection, green_pct[max_util_idx], ar[max_util_idx])
        util_array[max_util_idx] += runner.util(pop[max_util_idx], green_pct[max_util_idx], ng, pvty_pct[max_util_idx], reg)
        green_pct[max_util_idx] = ng
        funds -= injection
    return util_array, runner.geo_mean(util_array), util_delta

In [ ]:
x = np.ones(193)  # x is an array of utilities
green_pcts = df["green_percent"].to_numpy()
poverty_pcts = df["PctPvty"].to_numpy()
area = df.area  # array of areas
population = df.population  # array of populations
funds = 2000
reg = runner.regressor(df)

In [ ]:
f_util, f_mean, delta_prog = opt(x.copy(), green_pcts.copy(), area.copy(), population.copy(), poverty_pcts.copy(), funds, 10, reg)

In [ ]:
coords = pd.concat([df[['latitude', 'longitude']], pd.Series(f_util, name="util")], axis=1)
coords["density"] = df.population / df.area
coords.describe()

### Density Plot

In [ ]:
fig, ax = plt.subplots()
g = sns.scatterplot(data=coords, x="latitude", y="longitude", size="density", hue="density")
g.legend().remove()
g.set_title("Population Density")
norm = plt.Normalize(coords['density'].min(), coords['density'].max())
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
ax.figure.colorbar(sm).set_label("Population Density ($mi^2$)", rotation=270, labelpad=10)
cnst.sf('viz_density.png')

### Final Utility Plot

In [ ]:
fig, ax = plt.subplots()
cmap = sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True)
g = sns.scatterplot(data=coords, x="latitude", y="longitude", size="util", hue="util", palette=cmap)
g.legend().remove()
g.set_title("Final Tract Utility")
norm = plt.Normalize(coords['util'].min(), coords['util'].max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
ax.figure.colorbar(sm).set_label("Utility", rotation=270, labelpad=12)
cnst.sf('viz_f_utility.png')